In [1]:
import pickle

from data_fetching import fetch_solutions_batch, compute_split_solutions
from mechanism import (
    Trade,
    Solution,
    FilterRankRewardMechanism,
    NoFilter,
    BaselineFilter,
    DirectedTokenPairs,
    TokenPairs,
    TradedTokens,
    DirectSelection,
    MonotoneSelection,
    NoReward,
    SubsetFilteringSelection,
    ReferenceReward,
    SingleSurplusSelection,
    get_orders,
)

In [2]:
def compute_reward_statistic(solutions_batch, mechanisms, winners_rewards_batch):
    """Compute statistics of winners and rewards"""
    print(f"number of auction: {len(solutions_batch)}")

    # print(f"{len(mechanisms)} mechanisms tested: {mechanisms}")

    winners_batch, rewards_batch = zip(
        *[zip(*winners_rewards) for winners_rewards in winners_rewards_batch])

    print("\naverage number of winners per auction:")
    for k, _ in enumerate(mechanisms):
        print(
            f"{k}: {sum(len(winners[k]) for winners in winners_batch) / len(winners_batch)}"
        )

    print("\naverage total score per auction:")
    for k, _ in enumerate(mechanisms):
        print(
            f"{k}: {sum(sum(solution.score for solution in winners[k]) for winners in winners_batch) / len(winners_batch) / 10 ** 18}"
        )

    print("\naverage rewards per winner:")
    for k, _ in enumerate(mechanisms):
        print(
            f"{k}: {sum(sum(reward for reward, _ in rewards[k].values()) for rewards in rewards_batch) / sum(len(rewards[0]) for rewards in rewards_batch) / 10 ** 18}")

    print("\norder throughput:")
    for k, _ in enumerate(mechanisms):
        settled = sum(len(get_orders(winners[k])) for winners in
                      winners_batch)
        proposed = sum(len(get_orders(solutions)) for solutions in solutions_batch)
        print(
            f"{k}: {settled / proposed}")

    print("\nfrequency of differences between different mechanisms:")
    difference_matrix = {}
    for k in range(len(mechanisms)):
        for l in range(k + 1, len(mechanisms)):
            difference_matrix[(k, l)] = len(list(filter((lambda x: (
                        {solution.id for solution in x[k]} != {solution.id for solution in x[l]})),
                                                        winners_batch))) / len(winners_batch)
    print(difference_matrix)


def run_analysis(solutions_batch, mechanisms):
    """Run analysis for a batch of solutions

    Runs different mechanisms on the given batch of solutions and computes statistics.
    Parameters
    ----------
    solutions_batch : list[list[Solution]]
        List of solutions for each auction.
    mechanisms : list[AucitonMechanism]
        List of mechanisms to analyze on the given list of solutions.

    Returns
    -------
    all_winners_rewards: list[list[dict[str, tuple[int, int]]]]
        A list of winners and rewards for each auction and each mechanism.
    """
    all_winners_rewards: list[list[dict[str, tuple[int, int]]]] = []
    for i, solutions in enumerate(solutions_batch):
        winners_rewards = [mechanism.winners_and_rewards(solutions) for mechanism in mechanisms]

        all_winners_rewards.append(winners_rewards)

    compute_reward_statistic(solutions_batch, mechanisms, all_winners_rewards)

    return all_winners_rewards

Execute one of the following two cell.
1. A set of artificial examples
2. A set of historical auctions

In [7]:
# handcrafted examples
solutions_batch = [
    [  # batch vs single order solutions
        Solution(
            "batch winner",
            solver="solver 1",
            score=200,
            trades=[Trade("1", "A", "B", 100), Trade("2", "C", "D", 100)],
        ),
        Solution(
            "best on first trade",
            solver="solver 2",
            score=150,
            trades=[Trade("1", "A", "B", 150)],
        ),
        Solution(
            "best on second trade",
            solver="solver 3",
            score=150,
            trades=[Trade("2", "C", "D", 150)],
        ),
    ],
    [  # solutions without overlap
        Solution(
            "best on first trade",
            solver="solver 1",
            score=150,
            trades=[Trade("1", "A", "B", 150)],
        ),
        Solution(
            "best on second trade",
            solver="solver 2",
            score=140,
            trades=[Trade("2", "C", "D", 140)],
        ),
        Solution(
            "bad batch",
            solver="solver 3",
            score=100,
            trades=[Trade("1", "A", "B", 50), Trade("2", "C", "D", 50)],
        ),
    ],
    [  # batch in between solutions without overlap
        Solution(
            "best on first trade",
            solver="solver 1",
            score=150,
            trades=[Trade("1", "A", "B", 150)],
        ),
        Solution(
            "batch with overlap",
            solver="solver 3",
            score=100,
            trades=[Trade("1", "A", "B", 50), Trade("2", "C", "D", 50)],
        ),
        Solution(
            "best on second trade",
            solver="solver 2",
            score=90,
            trades=[Trade("2", "C", "D", 90)],
        ),
    ],
    [  # reference is not from winner
        Solution(
            "batch with overlap",
            solver="solver 1",
            score=200,
            trades=[Trade("1", "A", "B", 150), Trade("2", "C", "D", 50)],
        ),
        Solution(
            "best on first trade",
            solver="solver 1",
            score=100,
            trades=[Trade("1", "A", "B", 100)],
        ),
        Solution(
            "best on second trade",
            solver="solver 2",
            score=90,
            trades=[Trade("2", "C", "D", 90)],
        ),
    ],
    [  # token overlap but not on the same token pair
        Solution(
            "batch with overlap",
            solver="solver 1",
            score=100,
            trades=[Trade("1", "A", "B", 100)],
        ),
        Solution(
            "best on first trade",
            solver="solver 2",
            score=90,
            trades=[Trade("1", "A", "C", 90)],
        ),
    ],
    [
        Solution(
            id="batch winner",
            solver="solver 1",
            score=150,
            trades=[Trade("1", "A", "B", 100), Trade("2", "A", "C", 50)],
        ),
        Solution(
            id="unfair batch",
            solver="solver 2",
            score=110,
            trades=[Trade("1", "A", "B", 50), Trade("2", "A", "C", 60)],
        ),
        Solution(
            id="overlapping batch",
            solver="solver 3",
            score=100,
            trades=[Trade("3", "B", "A", 50), Trade("2", "A", "C", 50)],
        ),
        Solution(
            id="non-overlapping batch",
            solver="solver 4",
            score=100,
            trades=[Trade("3", "B", "A", 40), Trade("4", "D", "E", 60)],
        ),
        Solution(
            id="non-overlapping batch unfair",
            solver="solver 5",
            score=120,
            trades=[Trade("3", "B", "A", 20), Trade("4", "D", "E", 100)],
        ),
        Solution(
            id="reference A->B",
            solver="solver 1",
            score=80,
            trades=[Trade("1", "A", "B", 80)],
        ),
        Solution(
            id="reference A->C",
            solver="solver 2",
            score=40,
            trades=[Trade("2", "A", "C", 40)],
        ),
        Solution(
            id="runner up A->B",
            solver="solver 2",
            score=40,
            trades=[Trade("1", "A", "B", 40)],
        ),
        Solution(
            id="runner up A->C",
            solver="solver 1",
            score=40,
            trades=[Trade("2", "A", "C", 40)],
        ),
        Solution(
            id="reference B->A",
            solver="solver 7",
            score=30,
            trades=[Trade("3", "B", "A", 30)],
        ),
        Solution(
            id="reference F->G",
            solver="solver 8",
            score=50,
            trades=[Trade("5", "F", "G", 50)],
        ),
        Solution(
            id="runner up F->G",
            solver="solver 1",
            score=40,
            trades=[Trade("5", "F", "G", 40)],
        ),
        Solution(
            id="reference H->I",
            solver="solver 8",
            score=50,
            trades=[Trade("6", "H", "I", 50)],
        ),
    ],
]
solutions_batch_split = [compute_split_solutions(solutions) for solutions in solutions_batch]

In [8]:
# fetch auctions from file or database
# this can take around 20 minutes the first time it is run and creates a file of 80MB
auction_start = 10322553 - 50000
auction_end = 10322553
try:
    with open(f"batches_{auction_start}_{auction_end}.pickle", 'rb') as handle:
        solutions_batch = pickle.load(handle)
except FileNotFoundError:
    solutions_batch = fetch_solutions_batch(auction_start, auction_end)
    with open(f"batches_{auction_start}_{auction_end}.pickle", "wb") as handle:
        pickle.dump(solutions_batch, handle, protocol=-1)
solutions_batch_split = [compute_split_solutions(solutions) for solutions in solutions_batch]

In [9]:
filtering_function = DirectedTokenPairs()
mechanisms = [
    # our current mechanism
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(SingleSurplusSelection()),
        ReferenceReward(DirectSelection(SingleSurplusSelection()), 12 * 10 ** 15, 10 ** 16),
    ),
    # greedy choice of batches by surplus, same for references
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ),
        ReferenceReward(DirectSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ), 12 * 10 ** 15, 10 ** 16),
    ),
    # same as above but with fairness filtering
    FilterRankRewardMechanism(
        BaselineFilter(),
        DirectSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ),
        ReferenceReward(DirectSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ), 12 * 10 ** 15, 10 ** 16),
    ),
    # greedy choice of batches by surplus, in iteration checking for positive rewards
    FilterRankRewardMechanism(
        NoFilter(),
        MonotoneSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ),
        ReferenceReward(DirectSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ), 12 * 10 ** 15, 10 ** 16),
    ),
    # same as above but with fairness filtering
    FilterRankRewardMechanism(
        BaselineFilter(),
        MonotoneSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ),
        ReferenceReward(DirectSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ), 12 * 10 ** 15, 10 ** 16),
    )]

In [10]:
# compute results for submitted solutions (Step 1 (+ fairness))
all_results = run_analysis(solutions_batch, mechanisms)
# all_results

number of auction: 35501

average number of winners per auction:
0: 1.0
1: 1.4384383538491874
2: 1.5398721162784146
3: 1.4672544435368018
4: 1.5487451057716684

average total score per auction:
0: 0.19680579395853345
1: 0.1987690704639321
2: 0.19517176737881647
3: 0.1990946210279306
4: 0.1952787217530128

average rewards per winner:
0: 0.0021704975324772376
1: 0.002586540232696603
2: 0.0030776373853811387
3: 0.0028245165388456586
4: 0.003154884135275746

order throughput:
0: 0.6837831507170185
1: 0.8378315071701845
2: 0.8038024100148349
3: 0.8416443276162715
4: 0.8068474611560784

frequency of differences between different mechanisms:
{(0, 1): 0.31004760429283684, (0, 2): 0.4153967493873412, (0, 3): 0.32875130278020337, (0, 4): 0.41953747781752626, (1, 2): 0.1508689896059266, (1, 3): 0.02864708036393341, (1, 4): 0.15715050280273796, (2, 3): 0.13653136531365315, (2, 4): 0.009464522126137291, (3, 4): 0.13458775809132137}


In [11]:
# compute results for split submitted solutions (Step 2 + 3)
all_results = run_analysis(solutions_batch_split, mechanisms)

number of auction: 35501

average number of winners per auction:
0: 1.0
1: 1.7599222557111067
2: 2.171178276668263
3: 1.7958085687727106
4: 2.171178276668263

average total score per auction:
0: 0.19680579395853345
1: 0.20102419205330482
2: 0.20233549471582654
3: 0.20133661743490427
4: 0.20233549471582654

average rewards per winner:
0: 0.0021704975324772376
1: 0.0028952594367673526
2: 0.003220545462806234
3: 0.003241442892966044
4: 0.003220545462806234

order throughput:
0: 0.6834838508185201
1: 0.9864143872159904
2: 0.9871431174036385
3: 0.9839809489108087
4: 0.9871431174036385

frequency of differences between different mechanisms:
{(0, 1): 0.4446635306047717, (0, 2): 0.5481535731387849, (0, 3): 0.45821244471986705, (0, 4): 0.5481535731387849, (1, 2): 0.24945776175319004, (1, 3): 0.039322835976451365, (1, 4): 0.24945776175319004, (2, 3): 0.23954254809723668, (2, 4): 0.0, (3, 4): 0.23954254809723668}


More experiments

In [12]:
# super simple example
solutions_batch = [
    [
        Solution(
            id="batch winner",
            solver="solver 1",
            score=250,
            trades=[Trade("1", "A", "B", 150), Trade("2", "C", "D", 100)],
        ),
        Solution(
            id="overlapping batch",
            solver="solver 2",
            score=240,
            trades=[Trade("2", "C", "D", 140), Trade("3", "E", "F", 100)],
        ),
    ]
]
solutions_batch_split = [compute_split_solutions(solutions) for solutions in solutions_batch]

mechanisms = [
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(SingleSurplusSelection()),
        ReferenceReward(DirectSelection(SingleSurplusSelection()), 12 * 10 ** 15, 10 ** 16),
    ),
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(
            SubsetFilteringSelection(
                filtering_function=TradedTokens(), cumulative_filtering=False
            )
        ),
        ReferenceReward(DirectSelection(
            SubsetFilteringSelection(
                filtering_function=TradedTokens(), cumulative_filtering=False
            )
        ), 12 * 10 ** 15, 10 ** 16),
    ),
    FilterRankRewardMechanism(
        NoFilter(),
        MonotoneSelection(
            SubsetFilteringSelection(
                filtering_function=TradedTokens(), cumulative_filtering=False
            )
        ),
        ReferenceReward(DirectSelection(
            SubsetFilteringSelection(
                filtering_function=TradedTokens(), cumulative_filtering=False
            )
        ), 12 * 10 ** 15, 10 ** 16),
    ),
]

run_analysis(solutions_batch_split, mechanisms)

number of auction: 1

average number of winners per auction:
0: 1.0
1: 2.0
2: 2.0

average total score per auction:
0: 2.5e-16
1: 3.5e-16
2: 3.5e-16

average rewards per winner:
0: 1e-17
1: 2.1e-16
2: 2.1e-16

order throughput:
0: 0.6666666666666666
1: 1.0
2: 1.0

frequency of differences between different mechanisms:
{(0, 1): 1.0, (0, 2): 1.0, (1, 2): 0.0}


[[([Solution(id='batch winner', solver='solver 1', score=250, trades=[Trade(id='1', sell_token='A', buy_token='B', score=150), Trade(id='2', sell_token='C', buy_token='D', score=100)])],
   {'solver 1': (10, -240)}),
  ([Solution(id='batch winner', solver='solver 1', score=250, trades=[Trade(id='1', sell_token='A', buy_token='B', score=150), Trade(id='2', sell_token='C', buy_token='D', score=100)]),
    Solution(id="overlapping batch-('E', 'F')", solver='solver 2', score=100, trades=[Trade(id='3', sell_token='E', buy_token='F', score=100)])],
   {'solver 2': (100, -250), 'solver 1': (110, -240)}),
  ([Solution(id='batch winner', solver='solver 1', score=250, trades=[Trade(id='1', sell_token='A', buy_token='B', score=150), Trade(id='2', sell_token='C', buy_token='D', score=100)]),
    Solution(id="overlapping batch-('E', 'F')", solver='solver 2', score=100, trades=[Trade(id='3', sell_token='E', buy_token='F', score=100)])],
   {'solver 2': (100, -250), 'solver 1': (110, -240)})]]

In [ ]:
# comparison of overlap filtering
mechanisms = [
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(SingleSurplusSelection()),
        NoReward(),
    ),
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(
            SubsetFilteringSelection(
                filtering_function=TradedTokens(), cumulative_filtering=True
            )
        ),
        NoReward(),
    ),
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(
            SubsetFilteringSelection(
                filtering_function=TokenPairs(), cumulative_filtering=True
            )
        ),
        NoReward(),
    ),
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(
            SubsetFilteringSelection(
                filtering_function=DirectedTokenPairs(), cumulative_filtering=True
            )
        ),
        NoReward(),
    ),
]

In [ ]:
run_analysis(solutions_batch, mechanisms)